In [1]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [11]:
MicrosoftR = pd.read_csv(r"/Users/joakimlarsen/Documents/MasterOppgave/Microsoft/Microsoft_Ferdig.csv")

In [12]:
MicrosoftR.shape

(1812, 16)

In [13]:
MicrosoftR.head(3)

,Unnamed: 0,Date,Time,X.1,X,X_id,article_link,article_title,article_time,author_name,author_link,article_content,appears_in,symbols,related_articles,Time.1
0,2,2018-03-13,18:42:00,2,473,{'$oid': '5ab8d21e3efc2c03c520d00f'},https://www.nasdaq.com/article/is-oracle-orcl-...,Is Oracle (ORCL) Set to Beat Earnings Estimate...,{'$date': '2018-03-12T18:42:00Z'},"By Benjamin Rains, Zacks.com",https://www.nasdaq.com/author/zacks,Cloud computing has been one of the most well-...,"Investing,Earnings","AMZN,GOOGL,WDAY,MSFT,CRM",NaN,{'$date': '2018-03-12T18:42:00Z'}
1,4,2018-03-13,22:09:00,4,849,{'$oid': '5ab8d1dc3efc2c03c520cf8b'},https://www.nasdaq.com/article/1-smart-artific...,1 Smart Artificial Intelligence Bet You Can Ma...,{'$date': '2018-03-12T22:09:00Z'},"By Harsh Chauhan, Motley Fool",https://www.nasdaq.com/author/fool,"A report by IDC, which follows the tech indust...","PersonalFinance,Stocks","MSFT,AI,GOOGL,AMZN,GOOG",NaN,{'$date': '2018-03-12T22:09:00Z'}
2,6,2018-03-15,16:44:00,6,1313,{'$oid': '5ab8decb3efc2c03c520ec83'},https://www.nasdaq.com/article/3-large-cap-etf...,3 Large-Cap ETFs With Solid Inflows in March,{'$date': '2018-03-14T16:44:00Z'},"By Zacks Equity Research, Zacks.com",https://www.nasdaq.com/author/zacks,Wall Street has been recovering lately as fear...,"Investing,ETFs","AMZN,GOOGL,AAPL,MSFT,VV",NaN,{'$date': '2018-03-14T16:44:00Z'}


In [7]:
MicrosoftAksje = pd.read_csv(r"/Users/joakimlarsen/Documents/MasterOppgave/Microsoft/Microsoft_Prices_Label.csv")

In [8]:
MicrosoftAksje.shape

(8710, 8)

In [9]:
MicrosoftAksje.head(3)

,Date,Open,High,Low,Close,Adj Close,Volume,Label
0,1986-03-13,0.088542,0.101563,0.088542,0.097222,0.062055,1031788800,1
1,1986-03-14,0.097222,0.102431,0.097222,0.100694,0.064271,308160000,1
2,1986-03-17,0.100694,0.103299,0.100694,0.102431,0.065379,133171200,1


In [14]:
#Merge datasettene
merge = MicrosoftR.merge(MicrosoftAksje, how = "inner", on = "Date")

In [15]:
#Create a function to get the subjecticity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

#Create a fuction to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [16]:
#Create two new columns "Subjectivity" and "Polarity"
merge["Subjectivity"] = merge["article_content"].apply(getSubjectivity)
merge["Polarity"] = merge["article_content"].apply(getPolarity)

In [17]:
import nltk

from nltk.corpus import stopwords
stop = stopwords.words('english')

In [18]:
merge['article_content_without_stopwords'] = merge['article_content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [19]:
from nltk.stem import PorterStemmer

port = PorterStemmer()

In [20]:
def stem_words(article_content_without_stopwords):
    return " ".join([port.stem(word) for word in article_content_without_stopwords.split()])

merge["EtterStemming"] = merge["article_content_without_stopwords"].apply(lambda article_content_without_stopwords: stem_words(article_content_without_stopwords))
merge.head()

,Unnamed: 0,Date,Time,X.1,X,X_id,article_link,article_title,article_time,author_name,...,High,Low,Close,Adj Close,Volume,Label,Subjectivity,Polarity,article_content_without_stopwords,EtterStemming
0,2,2018-03-13,18:42:00,2,473,{'$oid': '5ab8d21e3efc2c03c520d00f'},https://www.nasdaq.com/article/is-oracle-orcl-...,Is Oracle (ORCL) Set to Beat Earnings Estimate...,{'$date': '2018-03-12T18:42:00Z'},"By Benjamin Rains, Zacks.com",...,97.239998,93.970001,94.410004,91.155731,35387800,0,0.475277,0.108063,Cloud computing one well-known drivers growth ...,cloud comput one well-known driver growth tech...
1,4,2018-03-13,22:09:00,4,849,{'$oid': '5ab8d1dc3efc2c03c520cf8b'},https://www.nasdaq.com/article/1-smart-artific...,1 Smart Artificial Intelligence Bet You Can Ma...,{'$date': '2018-03-12T22:09:00Z'},"By Harsh Chauhan, Motley Fool",...,97.239998,93.970001,94.410004,91.155731,35387800,0,0.440061,0.169724,"A report IDC, follows tech industry, estimates...","a report idc, follow tech industry, estim worl..."
2,10,2018-03-13,16:12:00,10,1660,{'$oid': '5ab8d2b83efc2c03c520d159'},https://www.nasdaq.com/article/3-strong-buy-se...,3 Strong Buy Semiconductor Stocks to Consider Now,{'$date': '2018-03-12T16:12:00Z'},"By Ryan McQueeney, Zacks.com",...,97.239998,93.970001,94.410004,91.155731,35387800,0,0.489353,0.146362,Semiconductor stocks battered recent market se...,semiconductor stock batter recent market sell-...
3,8,2018-03-13,07:05:00,8,1472,{'$oid': '5ab8d0693efc2c03c520cc37'},https://www.nasdaq.com/article/slack-needs-to-...,Slack Needs to Worry About Microsoft Teams,{'$date': '2018-03-13T07:05:00Z'},"By Timothy Green, Motley Fool",...,97.239998,93.970001,94.410004,91.155731,35387800,0,0.508590,0.175829,"Slack, software company focused workplace mess...","slack, softwar compani focus workplac messagin..."
4,6,2018-03-15,16:44:00,6,1313,{'$oid': '5ab8decb3efc2c03c520ec83'},https://www.nasdaq.com/article/3-large-cap-etf...,3 Large-Cap ETFs With Solid Inflows in March,{'$date': '2018-03-14T16:44:00Z'},"By Zacks Equity Research, Zacks.com",...,94.580002,92.830002,94.180000,90.933655,27611000,1,0.532905,0.115972,Wall Street recovering lately fears aggressive...,wall street recov late fear aggress rate hike ...


In [21]:
#Create a function to get the sentiment scores
def getSIA(EtterStemming):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(EtterStemming)
    return sentiment

In [23]:
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/joakimlarsen/nltk_data...


True

In [24]:
#Get the sentiment scores for each day
compound = []
neg = []
pos = []
neu = []
SIA = 0

for i in range(0, len(merge["EtterStemming"])):
    SIA = getSIA(merge["EtterStemming"][i])
    compound.append(SIA["compound"])
    neg.append(SIA["neg"])
    neu.append(SIA["neu"])
    pos.append(SIA["pos"])

In [25]:
#Store the sentiment scores in the merge data set:
merge["Compound"] = compound
merge["Negative"] = neg
merge["Neutral"] = neu
merge["Positive"] = pos

In [26]:
#Show the merge data
merge.head(3)

,Unnamed: 0,Date,Time,X.1,X,X_id,article_link,article_title,article_time,author_name,...,Volume,Label,Subjectivity,Polarity,article_content_without_stopwords,EtterStemming,Compound,Negative,Neutral,Positive
0,2,2018-03-13,18:42:00,2,473,{'$oid': '5ab8d21e3efc2c03c520d00f'},https://www.nasdaq.com/article/is-oracle-orcl-...,Is Oracle (ORCL) Set to Beat Earnings Estimate...,{'$date': '2018-03-12T18:42:00Z'},"By Benjamin Rains, Zacks.com",...,35387800,0,0.475277,0.108063,Cloud computing one well-known drivers growth ...,cloud comput one well-known driver growth tech...,0.9633,0.000,0.901,0.099
1,4,2018-03-13,22:09:00,4,849,{'$oid': '5ab8d1dc3efc2c03c520cf8b'},https://www.nasdaq.com/article/1-smart-artific...,1 Smart Artificial Intelligence Bet You Can Ma...,{'$date': '2018-03-12T22:09:00Z'},"By Harsh Chauhan, Motley Fool",...,35387800,0,0.440061,0.169724,"A report IDC, follows tech industry, estimates...","a report idc, follow tech industry, estim worl...",0.9892,0.048,0.823,0.130
2,10,2018-03-13,16:12:00,10,1660,{'$oid': '5ab8d2b83efc2c03c520d159'},https://www.nasdaq.com/article/3-strong-buy-se...,3 Strong Buy Semiconductor Stocks to Consider Now,{'$date': '2018-03-12T16:12:00Z'},"By Ryan McQueeney, Zacks.com",...,35387800,0,0.489353,0.146362,Semiconductor stocks battered recent market se...,semiconductor stock batter recent market sell-...,0.9948,0.016,0.844,0.140


In [27]:
#Create a list of column to keep
keep_columns = ["Open", "Low", "High", "Volume", "Subjectivity", "Polarity", "Compound", "Negative", "Neutral", "Positive", "Label"]
df = merge[keep_columns]
df

,Open,Low,High,Volume,Subjectivity,Polarity,Compound,Negative,Neutral,Positive,Label
0,97.000000,93.970001,97.239998,35387800,0.475277,0.108063,0.9633,0.000,0.901,0.099,0
1,97.000000,93.970001,97.239998,35387800,0.440061,0.169724,0.9892,0.048,0.823,0.130,0
2,97.000000,93.970001,97.239998,35387800,0.489353,0.146362,0.9948,0.016,0.844,0.140,0
3,97.000000,93.970001,97.239998,35387800,0.508590,0.175829,0.9906,0.062,0.803,0.135,0
4,93.529999,92.830002,94.580002,27611000,0.532905,0.115972,0.9877,0.074,0.781,0.145,1
...,...,...,...,...,...,...,...,...,...,...,...
1423,31.620001,31.250000,31.650000,63213000,0.461096,0.078854,0.9698,0.041,0.830,0.129,0
1424,32.869999,32.150002,32.869999,40685000,0.373844,-0.029014,0.7269,0.029,0.924,0.047,0
1425,37.240002,37.060001,38.220001,88948800,0.343695,0.045125,0.9300,0.040,0.867,0.094,1
1426,62.500000,62.240002,63.419998,35718900,0.400040,0.049240,0.7184,0.061,0.840,0.099,1


In [28]:
#Create the feature data set
X = df
X = np.array(X.drop(["Label"], 1))

#Create the target data set
y = np.array(df["Label"])

/var/folders/m7/01kjn8yx6d3_tjkcr1dkd9g80000gn/T/ipykernel_31133/988765945.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = np.array(X.drop(["Label"], 1))


In [45]:
#Split the data into 80% trainig and 20% testing data set
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [39]:
len(x_test)

286

In [40]:
len(x_train)

1142

In [30]:
#Create and train the model
model = LinearDiscriminantAnalysis().fit(x_train, y_train)

In [31]:
#Show the models predictions
predictions = model.predict(x_test)
predictions

array([0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0])

In [32]:
y_test

array([0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0])

In [33]:
#Show the model metrics
print( classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.80      0.87      0.83       140
           1       0.86      0.79      0.82       146

    accuracy                           0.83       286
   macro avg       0.83      0.83      0.83       286
weighted avg       0.83      0.83      0.83       286



In [46]:
#Create and train the model (Naive Bayes)
from sklearn.naive_bayes import GaussianNB
modelNB = GaussianNB()

In [55]:
#Train the model
modelNB.fit(x_train, y_train)

GaussianNB()

In [60]:
#Accuracy for modellen
modelNB.score(x_test, y_test)

0.5699300699300699

In [49]:
x_test[:10]

array([[ 5.78899990e+01,  5.68899990e+01,  5.80200000e+01,
         2.64974000e+07,  4.79085498e-01,  1.28204033e-01,
         9.91900000e-01,  4.70000000e-02,  8.19000000e-01,
         1.34000000e-01],
       [ 4.90200000e+01,  4.86699980e+01,  5.02400020e+01,
         4.67405000e+07,  5.16717931e-01,  1.35121896e-01,
         9.83900000e-01,  1.20000000e-02,  7.70000000e-01,
         2.18000000e-01],
       [ 5.04100000e+01,  4.98200000e+01,  5.04300000e+01,
         4.57105000e+07,  4.17602556e-01,  3.55148423e-02,
        -5.70700000e-01,  9.70000000e-02,  8.09000000e-01,
         9.40000000e-02],
       [ 5.49199980e+01,  5.34500010e+01,  5.49700010e+01,
         6.76255000e+07,  5.58333333e-01,  6.11111111e-02,
         2.96000000e-01,  0.00000000e+00,  9.37000000e-01,
         6.20000000e-02],
       [ 5.22999990e+01,  5.22500000e+01,  5.40700000e+01,
         1.35227100e+08,  5.58333333e-01,  6.11111111e-02,
         2.96000000e-01,  0.00000000e+00,  9.37000000e-01,
         6.

In [61]:
#Faktisk oppgang eller nedgang
y_test[:10]

array([0, 1, 0, 0, 1, 0, 0, 1, 0, 1])

In [62]:
#Sjekker om vi modellen predikerer rett
modelNB.predict(x_test[:10])

array([1, 0, 0, 0, 0, 0, 0, 1, 1, 1])

In [64]:
#Sjekker sannsynligheten for oppgang og nedgang
modelNB.predict_proba(x_test[:10])

array([[0.48253207, 0.51746793],
       [0.51222269, 0.48777731],
       [0.51009324, 0.48990676],
       [0.56930575, 0.43069425],
       [0.85480786, 0.14519214],
       [0.50292819, 0.49707181],
       [0.53514827, 0.46485173],
       [0.48087398, 0.51912602],
       [0.4777463 , 0.5222537 ],
       [0.48400922, 0.51599078]])